## **Dataset**

we will do data munging and analysis on a dataset about board games. The dataset is from Kaggle, at https://www.kaggle.com/andrewmvd/board-games. We have already downloaded the CSV file. After that, let's load the CSV file into a pandas DataFrame.

In [1]:
import pandas as pd

games = pd.read_csv('bgg_dataset.csv', delimiter=';', decimal=",")

We set `display.max_rows` to `None` for now so that we can see more values in code output. 

In [2]:
pd.set_option('display.max_rows', None)

Let's gain some basic understanding of the dataset by using `info()`, `head()`, and `describe()`.

In [3]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20343 entries, 0 to 20342
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  20327 non-null  float64
 1   Name                20343 non-null  object 
 2   Year Published      20342 non-null  float64
 3   Min Players         20343 non-null  int64  
 4   Max Players         20343 non-null  int64  
 5   Play Time           20343 non-null  int64  
 6   Min Age             20343 non-null  int64  
 7   Users Rated         20343 non-null  int64  
 8   Rating Average      20343 non-null  float64
 9   BGG Rank            20343 non-null  int64  
 10  Complexity Average  20343 non-null  float64
 11  Owned Users         20320 non-null  float64
 12  Mechanics           18745 non-null  object 
 13  Domains             10184 non-null  object 
dtypes: float64(5), int64(6), object(3)
memory usage: 2.2+ MB


In [4]:
games.head()

,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
0,174430.0,Gloomhaven,2017.0,1,4,120,14,42055,8.79,1,3.86,68323.0,"Action Queue, Action Retrieval, Campaign / Bat...","Strategy Games, Thematic Games"
1,161936.0,Pandemic Legacy: Season 1,2015.0,2,4,60,13,41643,8.61,2,2.84,65294.0,"Action Points, Cooperative Game, Hand Manageme...","Strategy Games, Thematic Games"
2,224517.0,Brass: Birmingham,2018.0,2,4,120,14,19217,8.66,3,3.91,28785.0,"Hand Management, Income, Loans, Market, Networ...",Strategy Games
3,167791.0,Terraforming Mars,2016.0,1,5,120,12,64864,8.43,4,3.24,87099.0,"Card Drafting, Drafting, End Game Bonuses, Han...",Strategy Games
4,233078.0,Twilight Imperium: Fourth Edition,2017.0,3,6,480,14,13468,8.70,5,4.22,16831.0,"Action Drafting, Area Majority / Influence, Ar...","Strategy Games, Thematic Games"


In [5]:
games.describe()

,ID,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users
count,20327.000000,20342.000000,20343.000000,20343.000000,20343.000000,20343.000000,20343.000000,20343.000000,20343.000000,20343.000000,20320.000000
mean,108216.245142,1984.249877,2.019712,5.672221,91.294548,9.601485,840.971391,6.403227,10172.890429,1.991188,1408.457628
std,98682.097298,214.003181,0.690366,15.231376,545.447203,3.645458,3511.562220,0.935911,5872.831597,0.848903,5040.179315
min,1.000000,-3500.000000,0.000000,0.000000,0.000000,0.000000,30.000000,1.050000,1.000000,0.000000,0.000000
25%,11029.000000,2001.000000,2.000000,4.000000,30.000000,8.000000,55.000000,5.820000,5087.500000,1.330000,146.000000
50%,88931.000000,2011.000000,2.000000,4.000000,45.000000,10.000000,120.000000,6.430000,10173.000000,1.970000,309.000000
75%,192939.500000,2016.000000,2.000000,6.000000,90.000000,12.000000,385.000000,7.030000,15258.500000,2.540000,864.000000
max,331787.000000,2022.000000,10.000000,999.000000,60000.000000,25.000000,102214.000000,9.580000,20344.000000,5.000000,155312.000000


## **Need for data cleaning and preprocessing**

The results of these several functions indicate a few needs for cleaning and preprocessing the data:

1) The columns `ID`, `Year Published` and `Owned Users` should be integers, but they are floating point numbers.

2) There are null values in various columns. 

3) It seems there could be wrong values. For instance, the minimal value in `Max Players` is 0. What kind of game is that if it allows at most zero player? 

4) The values in columns `Mechanics` and `Domains` are comma-separated lists. We need to parse these values and get the individual items from the lists.


Let's find out which columns have null values. This could be derived from the `Non-Null Count` in the output of `games.info()`. But there are much simpler ways.

In [6]:
# Code for Task 1
games.isnull().sum()

ID                       16
Name                      0
Year Published            1
Min Players               0
Max Players               0
Play Time                 0
Min Age                   0
Users Rated               0
Rating Average            0
BGG Rank                  0
Complexity Average        0
Owned Users              23
Mechanics              1598
Domains               10159
dtype: int64

Since column `ID` has null values, it couldn't be used for uniquely identifying games. Hence, let's take it out. 

In [7]:
games.drop('ID',axis=1,inplace=True)

The column `Year Published` has negative values, as `games.describe()` shows. It actually has 0 in its values too. Hence, we are using a year in the future (5000) to indicate the dataset doesn't provide the value for a game. The column `Owned Users` has 0s too. We thus use -1 to indicate missing values.

In [8]:
games.fillna(value={"Year Published":5000},inplace=True)

In [9]:
games.fillna(value={'Owned Users':-1},inplace=True)

In [10]:
# code block for converting the data type of column ``Year Pubblished`` to integer.
games = games.astype({'Year Published':'int16'})

In [11]:
# code block for converting the data type of column ``Owned Users`` to integer. 
games = games.astype({'Owned Users':'int32'})

After you finish Tasks 1 to 4, run `games.info()`, `games.head()`, and `games.describe()` again, to verify you have achieved the goals. In fact, you could do this from time to time, in various places, to make sure you haven't messed up the data.

In [12]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20343 entries, 0 to 20342
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Name                20343 non-null  object 
 1   Year Published      20343 non-null  int16  
 2   Min Players         20343 non-null  int64  
 3   Max Players         20343 non-null  int64  
 4   Play Time           20343 non-null  int64  
 5   Min Age             20343 non-null  int64  
 6   Users Rated         20343 non-null  int64  
 7   Rating Average      20343 non-null  float64
 8   BGG Rank            20343 non-null  int64  
 9   Complexity Average  20343 non-null  float64
 10  Owned Users         20343 non-null  int32  
 11  Mechanics           18745 non-null  object 
 12  Domains             10184 non-null  object 
dtypes: float64(2), int16(1), int32(1), int64(6), object(3)
memory usage: 1.8+ MB


In [13]:
games.head()

,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
0,Gloomhaven,2017,1,4,120,14,42055,8.79,1,3.86,68323,"Action Queue, Action Retrieval, Campaign / Bat...","Strategy Games, Thematic Games"
1,Pandemic Legacy: Season 1,2015,2,4,60,13,41643,8.61,2,2.84,65294,"Action Points, Cooperative Game, Hand Manageme...","Strategy Games, Thematic Games"
2,Brass: Birmingham,2018,2,4,120,14,19217,8.66,3,3.91,28785,"Hand Management, Income, Loans, Market, Networ...",Strategy Games
3,Terraforming Mars,2016,1,5,120,12,64864,8.43,4,3.24,87099,"Card Drafting, Drafting, End Game Bonuses, Han...",Strategy Games
4,Twilight Imperium: Fourth Edition,2017,3,6,480,14,13468,8.70,5,4.22,16831,"Action Drafting, Area Majority / Influence, Ar...","Strategy Games, Thematic Games"


In [14]:
games.describe()

,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users
count,20343.000000,20343.000000,20343.000000,20343.000000,20343.000000,20343.000000,20343.000000,20343.000000,20343.000000,20343.000000
mean,1984.398122,2.019712,5.672221,91.294548,9.601485,840.971391,6.403227,10172.890429,1.991188,1406.864081
std,215.039951,0.690366,15.231376,545.447203,3.645458,3511.562220,0.935911,5872.831597,0.848903,5037.551824
min,-3500.000000,0.000000,0.000000,0.000000,0.000000,30.000000,1.050000,1.000000,0.000000,-1.000000
25%,2001.000000,2.000000,4.000000,30.000000,8.000000,55.000000,5.820000,5087.500000,1.330000,146.000000
50%,2011.000000,2.000000,4.000000,45.000000,10.000000,120.000000,6.430000,10173.000000,1.970000,308.000000
75%,2016.000000,2.000000,6.000000,90.000000,12.000000,385.000000,7.030000,15258.500000,2.540000,864.000000
max,5000.000000,10.000000,999.000000,60000.000000,25.000000,102214.000000,9.580000,20344.000000,5.000000,155312.000000


Earlier we noticed the existence of value 0 in certain columns which shouldn't have such values. Let's find out how prevalent the problem is. 

In [15]:
(games == 0).sum()

Name                     0
Year Published         185
Min Players             46
Max Players            161
Play Time              556
Min Age               1251
Users Rated              0
Rating Average           0
BGG Rank                 0
Complexity Average     426
Owned Users              1
Mechanics                0
Domains                  0
dtype: int64

If you get the correct code, you will see from the output that 46 rows have value 0 in column `Min Players`, 161 in column `Max Players`, 185 in `Year Published`, and so on. We need to keep this mind when we analyze the data so that we don't draw inaccurate conclusions. 

Particularly, let's examine `Year Published`. We discovered earlier that it also has negative values. We can take a closer look now.



you will find that Year 2017 has 1274 games published, which is the most among all years. 

In [16]:
games['Year Published'].value_counts()

 2017    1274
 2016    1257
 2018    1254
 2019    1134
 2015    1131
 2014     987
 2013     850
 2012     815
 2011     735
 2010     692
 2020     684
 2009     631
 2008     580
 2005     544
 2007     522
 2006     516
 2004     490
 2003     408
 2002     335
 2001     298
 2000     295
 1999     268
 1998     246
 1997     221
 1996     216
 1995     212
 1992     201
 0        185
 1993     183
 1994     180
 1991     175
 1986     149
 1981     149
 1990     146
 2021     144
 1987     136
 1989     132
 1983     130
 1985     129
 1988     128
 1979     126
 1980     125
 1982     124
 1975     113
 1977     102
 1978      97
 1984      96
 1973      73
 1974      69
 1972      66
 1976      64
 1971      36
 1970      33
 1965      26
 1969      25
 1968      24
 1967      20
 1964      19
 1960      19
 1962      18
 1963      17
 1959      11
 1961      11
 1930      11
 1966      10
 1956       9
 1958       9
 1935       7
 1954       7
 1938       6
 1940       6
 1955 

The oldest game was from 3500 BC. Let's find out which game it is. 

In [17]:
games.loc[games['Year Published'] == games['Year Published'].min()].values[0][0]

'Senet'

you see that in general the number of games published in a year has been steadily increasing. However, Year 0 appears to be an outlier, as it has 185 games according to the dataset. This doesn't seem right.

0 was used to indicate unknown/missing publishing year when the dataset was created. How confusing that is. This reminds us it is important to make good choices in dealing with missing values. 

In [18]:
len(games[(games['Year Published']<1900) & (games['Year Published']!=0)])

111

For the same reason, we believe value 0 in all other columns are not reliable either. Our next task will replace 5000 in `Year Published` and -1 in `Owned Users` by 0. Remember they were actually null values. Later we will ignore them together with all 0 values in our analysis.

In [19]:
games.replace(to_replace={'Year Published':5000,'Owned Users':-1},value=0,inplace=True)

Also, let's take a look at the frequency of each value in column `Min Players`.

In [20]:
pd.Series({0: 46, 1: 3270, 2: 14076, 3: 2365, 4: 474, 5: 57, 6: 21, 7: 14, 8: 17, 9: 1, 10: 2}, name = 'Min Players')

0        46
1      3270
2     14076
3      2365
4       474
5        57
6        21
7        14
8        17
9         1
10        2
Name: Min Players, dtype: int64

Our next task attempts to examine the relationship between `Min Players` and popularity of games measured by ownership. 

As we discussed, we don't trust the value 0 in any of the columns. We can thus ignore the games with 0 in `Min Players`. Furthermore, there are only 1 and 2 games for `Min Players` 9 and 10, respectively. The statistics of these games won't be meaningful. When we focus on the rest of the games in the output of Task 10, we will observe a general pattern of decresing ownership by minimum required players. That is probably not surprising, since it is easier to find people to play games with less required players.  

In [21]:
games[(games['Min Players']!=0)&(games['Min Players']!=9)&(games['Min Players']!=10)&(games['Owned Users']!=0)].groupby(['Min Players'])['Owned Users'].mean()

Min Players
1    1762.693909
2    1348.215520
3    1309.574756
4    1218.301688
5    2827.526316
6    1154.761905
7     332.571429
8    1661.705882
Name: Owned Users, dtype: float64

The pattern appears to have some exceptions, in games with `Min Players` being 5 and 8. Let's take a further look. Before we continue, let's change `display.max_rows` to 50.

In [22]:
pd.set_option('display.max_rows', 50)

In [23]:
games.sort_values(by=['Owned Users'],ascending=False).groupby(['Min Players']).head()

,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
98,Pandemic,2008,2,4,45,8,102214,7.61,99,2.41,155312,"Action Points, Cooperative Game, Hand Manageme...","Family Games, Strategy Games"
394,Catan,1995,3,4,120,10,101510,7.15,395,2.32,154531,"Dice Rolling, Hexagon Grid, Income, Modular Bo...","Family Games, Strategy Games"
177,Carcassonne,2000,2,5,45,7,101853,7.42,178,1.91,149337,"Area Majority / Influence, Map Addition, Tile ...",Family Games
60,7 Wonders,2010,2,7,30,10,84371,7.75,61,2.33,112410,"Card Drafting, Drafting, Hand Management, Set ...","Family Games, Strategy Games"
92,Codenames,2015,2,8,15,14,67688,7.62,93,1.29,107682,"Communication Limits, Memory, Push Your Luck, ...",Party Games
97,Dominion,2008,2,4,30,13,78089,7.62,98,2.36,101839,"Deck Bag and Pool Building, Delayed Purchase, ...",Strategy Games
3,Terraforming Mars,2016,1,5,120,12,64864,8.43,4,3.24,87099,"Card Drafting, Drafting, End Game Bonuses, Han...",Strategy Games
30,Agricola,2007,1,5,150,12,63498,7.94,31,3.64,75679,"Advantage Token, Automatic Resource Growth, Ca...",Strategy Games
13,Scythe,2016,1,5,115,14,57871,8.24,14,3.41,75640,"Area Majority / Influence, Card Play Conflict ...",Strategy Games
4401,Munchkin,2001,3,6,120,10,41605,5.90,4403,1.80,73910,"Hand Management, Take That, Variable Player Po...",Thematic Games


The output of `games.describe()` shows that column `Complexity Average` has value 0. Based on our earlier analysis, we shouldn't put much faith in this value. Besides 0, the smallest value in that column is 1. Therefore we decide to have four bins for `Complexity Average`. 

In [24]:
games['Rating_Average'] = pd.cut(games['Rating Average'],[0.999,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0])
games['Complexity_Average'] = pd.cut(games['Complexity Average'],[0.999,2.0,3.0,4.0,5.0])
games.pivot_table(values='Name',index='Rating_Average',columns='Complexity_Average',aggfunc='count')

Complexity_Average,"(0.999, 2.0]","(2.0, 3.0]","(3.0, 4.0]","(4.0, 5.0]"
Rating_Average,,,,
"(0.999, 2.0]",6.0,2.0,NaN,NaN
"(2.0, 3.0]",19.0,4.0,1.0,NaN
"(3.0, 4.0]",154.0,16.0,3.0,1.0
"(4.0, 5.0]",1023.0,101.0,18.0,2.0
"(5.0, 6.0]",3972.0,860.0,127.0,8.0
"(6.0, 7.0]",4856.0,2906.0,650.0,57.0
"(7.0, 8.0]",1209.0,1978.0,1029.0,163.0
"(8.0, 9.0]",102.0,276.0,256.0,88.0
"(9.0, 10.0]",9.0,9.0,9.0,3.0


The pivot table suggests a positive correlation between these two columns. As the complexity of games increases, the rating also tends to increase. Perhaps this is intuitive. For complex games to have a market, it needs to be of higher quality. 

In fact, we can directly calculate the correlation using `corr`, as follows. The value of 0.5 in Pearson correlation coefficient suggests a fairly large positive correlation.

In [25]:
g = games[games['Complexity Average']>0]

g['Rating Average'].corr(g['Complexity Average'], method='pearson')

0.510870853513205

Next, we are going to produce a similar pivot table, focusing on `Owned Users`. Ideally we want to exclude the games with value 0 on this column. To simplify things, we are not requiring you to do it. The pattern we will be seeing is not changed, since only 23 games would have been removed. 

In [26]:
games['Rating_Average'] = pd.cut(games['Rating Average'],[0.999,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0])
games['Complexity_Average'] = pd.cut(games['Complexity Average'],[0.999,2.0,3.0,4.0,5.0])
games.pivot_table(values='Owned Users',index='Rating_Average',columns='Complexity_Average',aggfunc=('mean'))

Complexity_Average,"(0.999, 2.0]","(2.0, 3.0]","(3.0, 4.0]","(4.0, 5.0]"
Rating_Average,,,,
"(0.999, 2.0]",21.000000,90.000000,NaN,NaN
"(2.0, 3.0]",526.736842,119.750000,133.000000,NaN
"(3.0, 4.0]",364.188312,234.125000,207.666667,176.000000
"(4.0, 5.0]",436.129032,232.287129,240.444444,71.000000
"(5.0, 6.0]",539.152568,485.956977,388.976378,321.000000
"(6.0, 7.0]",1150.022446,1091.727116,885.712308,811.631579
"(7.0, 8.0]",3296.243176,3273.025784,2928.022352,1832.558282
"(8.0, 9.0]",734.127451,2583.463768,4430.777344,3669.193182
"(9.0, 10.0]",62.222222,66.555556,75.666667,196.000000


this pivot table also shows some interesting patterns. At every rating tier till (7, 8], simpler games enjoy larger ownerships. However, for the really good games with ratings greater than 8, players are not afraid of their complexity. In fact, the more complex games in this rating tier get owned by more players.

Now we will process the `Mechanics` and `Domains` columns. They store values as strings. Each string is a comma-separated list of items. The following code will turn `Mechanics` into a DataFrame itself, with each column corresponding to a unique item from the comma-separated lists. Similarly, we are creating a new DataFrame for the `Domains` column.

In [27]:
mechanics = games['Mechanics'].str.get_dummies(sep=", ")
domains = games['Domains'].str.get_dummies(sep=", ")

mechanics = pd.concat([games['Name'], mechanics], 1)
domains = pd.concat([games['Name'], domains], 1)

Let's take a look at the columns in the new DataFrame `mechanics`. The values are 1 and 0, i.e., essentially Boolean, indicating whether a game uses the corresponding mechanics or not. This is also called *one-hot encoding*. 


In [28]:
mechanics.head()

,Name,Acting,Action Drafting,Action Points,Action Queue,Action Retrieval,Action Timer,Action/Event,Advantage Token,Alliances,Area Majority / Influence,Area Movement,Area-Impulse,Auction/Bidding,Auction: Dexterity,Auction: Dutch,Auction: Dutch Priority,Auction: English,Auction: Fixed Placement,Auction: Once Around,Auction: Sealed Bid,Auction: Turn Order Until Pass,Automatic Resource Growth,Betting and Bluffing,Bias,Bingo,Bribery,Campaign / Battle Card Driven,Card Drafting,Card Play Conflict Resolution,Catch the Leader,Chaining,Chit-Pull System,Closed Economy Auction,Command Cards,Commodity Speculation,Communication Limits,Connections,Constrained Bidding,Contracts,...,Simultaneous Action Selection,Singing,Single Loser Game,Slide/Push,Solo / Solitaire Game,Speed Matching,Square Grid,Stacking and Balancing,Stat Check Resolution,Static Capture,Stock Holding,Storytelling,Sudden Death Ending,Take That,Targeted Clues,Team-Based Game,Tech Trees / Tech Tracks,Three Dimensional Movement,Tile Placement,Time Track,Track Movement,Trading,Traitor Game,Trick-taking,Tug of War,Turn Order: Auction,Turn Order: Claim Action,Turn Order: Pass Order,Turn Order: Progressive,Turn Order: Random,Turn Order: Role Order,Turn Order: Stat-Based,Variable Phase Order,Variable Player Powers,Variable Set-up,Victory Points as a Resource,Voting,Worker Placement,Worker Placement with Dice Workers,Zone of Control
0,Gloomhaven,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,Pandemic Legacy: Season 1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Brass: Birmingham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
3,Terraforming Mars,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
4,Twilight Imperium: Fourth Edition,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0


Similarly the new DataFrame `domains` uses one-hot encoding to record the games' domain types. 

In [29]:
domains.head()

,Name,Abstract Games,Children's Games,Customizable Games,Family Games,Party Games,Strategy Games,Thematic Games,Wargames
0,Gloomhaven,0,0,0,0,0,1,1,0
1,Pandemic Legacy: Season 1,0,0,0,0,0,1,1,0
2,Brass: Birmingham,0,0,0,0,0,1,0,0
3,Terraforming Mars,0,0,0,0,0,1,0,0
4,Twilight Imperium: Fourth Edition,0,0,0,0,0,1,1,0


Let's find out which are the most common game domains and which are the least common ones. there are 3316 `Wargames`, the most common type. The least common type is `Customizable Games`, with 297 games. 

In [30]:
games = games.assign(Domains=games['Domains'].str.split(',')).explode('Domains')
games['Domains'].apply(lambda x:str(x).strip()).value_counts().sort_index()[:-1]

Abstract Games        1070
Children's Games       849
Customizable Games     297
Family Games          2173
Party Games            605
Strategy Games        2205
Thematic Games        1174
Wargames              3316
Name: Domains, dtype: int64

In [31]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21848 entries, 0 to 20342
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   Name                21848 non-null  object  
 1   Year Published      21848 non-null  int16   
 2   Min Players         21848 non-null  int64   
 3   Max Players         21848 non-null  int64   
 4   Play Time           21848 non-null  int64   
 5   Min Age             21848 non-null  int64   
 6   Users Rated         21848 non-null  int64   
 7   Rating Average      21848 non-null  float64 
 8   BGG Rank            21848 non-null  int64   
 9   Complexity Average  21848 non-null  float64 
 10  Owned Users         21848 non-null  int32   
 11  Mechanics           20189 non-null  object  
 12  Domains             11689 non-null  object  
 13  Rating_Average      21848 non-null  category
 14  Complexity_Average  21422 non-null  category
dtypes: category(2), float64(2), int16(1)

In [32]:
games[(games['Domains']=="Wargames") & (games['Complexity Average']!=0)]['Complexity Average'].mean()

2.8936339610175064

If you perform the same task on `Children's Games`, you will get 1.175. These two are the two types of games with the largest and smallest average `Complexity Average`. If you want to calculate this for every type of games, the following code does it.

In [33]:
results = \
    (games[games['Complexity Average']>0].set_index(games.columns.drop('Domains',1).tolist())
    .Domains.str.split(', ', expand=True)
    .stack()
    .reset_index()
    .rename(columns={0:'domain'})
    .loc[:,['domain','Owned Users', 'Rating Average', 'Complexity Average']]
    .groupby('domain').agg({'Owned Users':['mean'], 'Rating Average':['mean'], 'Complexity Average':['mean']})
    )
results 

,Owned Users,Rating Average,Complexity Average
,mean,mean,mean
domain,,,
Children's Games,750.708333,5.332083,1.286667
Customizable Games,546.500000,5.560000,2.195000
Family Games,3532.273543,6.236726,1.471749
Party Games,3610.842105,6.214408,1.292368
Strategy Games,4765.240093,6.664056,2.166084
Thematic Games,4057.963918,6.687500,2.408943
Wargames,2145.299652,6.798641,2.671498
Abstract Games,1362.797170,6.228330,1.968028
